In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./data/AVAX-USD.csv')

In [3]:
df

,Close,High,Low,Transactions,Open,Timestamp,Volume,AVG_Price_Volume
0,57.520,58.320,56.31,6051,56.970,2022-05-01 00:00:00,74036.002080,57.5356
1,58.480,58.800,56.48,4948,57.478,2022-05-01 01:00:00,64954.450505,57.8494
2,57.805,58.890,57.77,2964,58.480,2022-05-01 02:00:00,34858.409040,58.3877
3,58.710,58.710,57.71,2532,57.780,2022-05-01 03:00:00,26857.805865,58.2918
4,58.210,58.750,57.75,2507,58.720,2022-05-01 04:00:00,35553.619278,58.2732
...,...,...,...,...,...,...,...,...
1195,15.580,15.890,15.40,3042,15.770,2022-06-19 19:00:00,138864.451408,15.5890
1196,16.290,16.341,15.57,3339,15.570,2022-06-19 20:00:00,139554.446623,16.0654
1197,16.160,16.310,16.07,1466,16.290,2022-06-19 21:00:00,42074.643573,16.2025
1198,16.140,16.340,16.01,1603,16.160,2022-06-19 22:00:00,62234.236247,16.1514


In [4]:
df.describe()

,Close,High,Low,Transactions,Open,Volume,AVG_Price_Volume
count,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
mean,31.644437,32.134882,31.227987,3523.797500,31.677787,91930.632563,31.666926
std,13.799495,13.934998,13.683125,3233.249591,13.811928,81817.320745,13.805369
min,13.880000,14.090000,13.740000,593.000000,13.860000,6932.728746,13.899600
25%,23.345000,23.643500,23.057500,1692.000000,23.350000,38956.165870,23.304075
50%,27.620000,28.023000,27.250000,2588.000000,27.645000,69104.352224,27.618500
75%,34.025000,34.631750,33.342500,4160.250000,34.040000,119347.169414,33.955025
max,68.580000,69.480000,68.080000,38211.000000,68.590000,898444.650317,68.661500


In [5]:
def task_1_1 (df:pd.DataFrame) -> pd.DataFrame:
        
    """
    На вход подается pd.Dataframe с данными рынка криптовалюты AVAX
    где производятся вычисления согласно задаче
    1.1) Если в период роста цена падает менее, чем на 0,6%, и не ниже точки входа, а затем опять поднимается выше предыдущего максимума,
    то мы не выходим из промежутка, а считаем этот небольшой период падения частью периода роста.
    ---------------------------------------------------------------------------------------
    Returns
    df: pd.Dataframe
        датафрейм с флагом и вспомогательными вычислениями 
    """
    df['delta_close'] = df['Close'] - df['Open'].shift(1)                                 ### период роста
    
    df['0.6_percente'] = (df['AVG_Price_Volume'] * 0.6) / 100                             ### расчет 0.6 процентов
    
    df['delta_close_avg'] = df['Close'] - df['AVG_Price_Volume']                          ### разница в цене и закрытии
    
    df['delta_close_below_zero'] = np.where(df['delta_close_avg'] < 0, 1, 0)              ### флаг где дельта разници цены и закрытия отрицательна

    df['below_zero_low_like_0.6_percemte'] = np.where((df['delta_close_below_zero'] == 1) ### флаг где дельта закрытия отрицательна
        & (df['delta_close_avg'].abs() < df['0.6_percente']),1, 0)                        ### и разница в цене меньше 0.6 процентов  

    df['flag_1.1'] = np.where(                                                            ### флаг для задачи 1.1
        (df['delta_close'] > 0)                                                           ### период роста 
        & (df['below_zero_low_like_0.6_percemte'] == 1)                                   ### где падение цены ниже 0.6 процентов
        & (df['AVG_Price_Volume'] > df['Open'])                                           ### цена выше точки входа 
        & (df['AVG_Price_Volume'] > df['High'].shift(-1)), 1, 0)                          ### и а затем выше предидущего максимума
    
    return df

In [6]:
def task_1_2 (df:pd.DataFrame) -> pd.DataFrame:
    """
    На вход подается pd.Dataframe с данными рынка криптовалюты AVAX и вспомогательными вычислениями из функции task_1.1
    где производятся вычисления согласно задаче
    1.2) Если доход в цене за некоторый период роста <1%, то данный период не считается доходным;
    ---------------------------------------------------------------------------------------
    Returns
    df: pd.Dataframe
        датафрейм с флагом и вспомогательными вычислениями 
    """
    
    df['1_percente'] = (df['AVG_Price_Volume'] * 1) / 100                                 ### расчет 1 процента
    
    df['flag_1.2'] = np.where(                                                            ### флаг для задачи 1.2
        (df['delta_close'] > 0)                                                           ### период роста
        & (df['delta_close'] < df['1_percente']), 1, 0)                                   ### если доход в цене за некоторый период роста <1%
    
    return df

In [7]:
def task_2 (df:pd.DataFrame, time_trashhold:str) -> float:
    
    """
    На вход подается pd.Dataframe с уже выполнеными вычислениямми из функций task_1_1, task_1_2 и временной фильтр согласно задаче 
    2) Подсчитываем суммарную доходность за весь месяц, складывая доходности всех полученных промежутков роста.
    ---------------------------------------------------------------------------------------
    Returns
    result: float64
        сумма дохода за указанный месяц с учетом указанных в задаче точкек роста
    """
    
    
    df_with_task_filtre = df.loc[((df['delta_close'] > 0)                               ### период роста 
        | (df['flag_1.1'] == 1))                                                        ### период роста из задачи 1.1
        & (df['flag_1.2'] == 0)                                                         ### не включая период роста задачи 1.2
        & (df['Timestamp'] >= time_trashhold)                                           ### за май месяц
      ]
    
    result = (df_with_task_filtre['delta_close'] * df_with_task_filtre['Volume']).sum() ### сумма дохода за месяц с учетом всех фильтров 
    
    return print(f'доходность за весь месяц {result.round(2)} usd ')                    ### вывод на экран суммы прибыли за месяц

In [8]:
df = task_1_1(df)

df = task_1_2(df)

time_trashhold = '2022-06-01'
task_2(df, time_trashhold)

доходность за весь месяц 9034994.86 usd 
